<a href="https://colab.research.google.com/github/kikun1234/Medium_Python_Stats/blob/main/Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
##importing libraries & import google sheets
import pandas as pd
import numpy as np

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

ws = gc.open('Stats').get_worksheet(1)
df = pd.DataFrame(ws.get_all_records())

In [36]:
df1 = df[~df['Title'].str.contains('May 2021|February 2022|March 2022|April 2022')]
df1['Title'] = df1['Title'].shift(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
df2 = df1.reset_index(drop=True)
df2 = df2.iloc[1:]
df2= df2[~df2['Title'].str.contains('storyDetails')]

In [38]:
def clean(x):
  df4 = x.str.split('([^\W\d])', expand=True)
  df4 = df4.iloc[: , :-1]
  df4.columns=["A","B"]
  df4['B'] = df4['B'].str.replace("K","1000")
  df4['A'] = df4['A'].astype('float64')
  df4['B'] = df4['B'].astype('float64')
  df4['C'] = df4['A']*df4['B']
  return(df4['C'])

In [39]:
df3 = df2
views1 = clean(df2['Views'])
df3["Views1"] = views1
df3['Views1'] = df3['Views1'].fillna(df3['Views'])

reads1 = clean(df2['Reads'])
df3["Reads1"] = reads1
df3['Reads1'] = df3['Reads1'].fillna(df3['Reads'])

df3['Read Ratio']=df3['Read Ratio'].str.replace('\%','')
df3['Read Ratio'] = df3['Read Ratio'].astype('float64')
df3['Read Ratio'] = df3['Read Ratio']/100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


In [40]:
df4=df3
df4['Views2']=df4['Reads1']/df4['Read Ratio']
df4['Views2'] = df4['Views2'].round(0)
df4['Fans'] = df4['Fans'].astype('float64')
df_stat1 = df4[['Title','Views2','Reads1','Read Ratio','Fans']]

In [42]:
ws = gc.open('Stats').get_worksheet(0)
df = pd.DataFrame(ws.get_all_records())

In [43]:
df['Title'] = df['Title'].replace('', np.nan) 
df1 = df.dropna()
df1 = df1[df1['Title'].str.len()<100]
df1 = df1[~df1['Title'].str.contains('Published in|Get an email whenever Jason Huynh publishes.|You cannot subscribe to yourself')]
#df1 = df1.reset_index(drop=True)
df2 = df1
df2 = df2[df2['Title'].str.len()>20]
#df2 = df2.drop_duplicates()
titles = df2.iloc[::2].reset_index(drop=True)

In [44]:
df4 = df1[df1['Title'].str.len()<20]
date = df4[::3].reset_index(drop=True)
tag = df4[1::3].reset_index(drop=True)
time = df4[2::3].reset_index(drop=True)

In [45]:
df_stats2 = pd.concat([titles,date, tag, time],axis=1)
df_stats2.columns = ['Title','Date', 'Tag', 'Time']
df_stats2['Date'] = df_stats2['Date'].str.replace('\·','')
df_stats2['Time'] = df_stats2['Time'].str.replace('min read','')
df_stats2['Time'] = df_stats2['Time'].astype("float64")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
df_final = df_stat1.merge(df_stats2,on="Title")